In [1]:
import pandas as pd
import glob
import os
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv('../summary_data/original/all_tools_TRB.csv')
data

,Sample,CDR3,nReads_TCR,nReads_MIXCR,nReads_IMREP,nReads_TRUST4,tissue,tissue_type,total_reads_TCR,total_reads_MIXCR,total_reads_IMREP,total_reads_TRUST4,frequency_TCR,frequency_MIXCR,frequency_IMREP,frequency_TRUST4,repertoire_type,class
0,SRR5233637,CASSLPKETQYF,11.0,0.0,0.0,0.0,small_intestine,T_cell_poor,3047629.0,84.0,315.0,544.0,0.000004,0.0,0.0,0.000000,polyclonal,T_cell_poor_polyclonal
1,SRR5233637,CASLGRHYNEQFF,1097.0,0.0,0.0,0.0,small_intestine,T_cell_poor,3047629.0,84.0,315.0,544.0,0.000360,0.0,0.0,0.000000,polyclonal,T_cell_poor_polyclonal
2,SRR5233637,CASTQAAGTEKLFF,42.0,0.0,0.0,0.0,small_intestine,T_cell_poor,3047629.0,84.0,315.0,544.0,0.000014,0.0,0.0,0.000000,polyclonal,T_cell_poor_polyclonal
3,SRR5233637,CASSVPRGDYEQYF,61.0,0.0,0.0,0.0,small_intestine,T_cell_poor,3047629.0,84.0,315.0,544.0,0.000020,0.0,0.0,0.000000,polyclonal,T_cell_poor_polyclonal
4,SRR5233637,CASTPLGAEGSPLHF,2994.0,0.0,0.0,0.0,small_intestine,T_cell_poor,3047629.0,84.0,315.0,544.0,0.000982,0.0,0.0,0.000000,polyclonal,T_cell_poor_polyclonal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389052,sample15,CASSLGGAGGYAF,0.0,0.0,0.0,2.0,melanoma,T_cell_poor,749686.0,113.0,152.0,351.0,0.000000,0.0,0.0,0.005698,polyclonal,T_cell_poor_polyclonal
389053,sample15,CASS?FGAGASGYTDTQYF,0.0,0.0,0.0,2.0,melanoma,T_cell_poor,749686.0,113.0,152.0,351.0,0.000000,0.0,0.0,0.005698,polyclonal,T_cell_poor_polyclonal
389054,sample15,CASSWTGSQETQYF,0.0,0.0,0.0,2.0,melanoma,T_cell_poor,749686.0,113.0,152.0,351.0,0.000000,0.0,0.0,0.005698,polyclonal,T_cell_poor_polyclonal
389055,sample15,CASSVEGYEQYF,0.0,0.0,0.0,2.0,melanoma,T_cell_poor,749686.0,113.0,152.0,351.0,0.000000,0.0,0.0,0.005698,polyclonal,T_cell_poor_polyclonal


In [3]:
def compute_frequency(samples_data, tool, discard_zero_freq_samples):

    samples = set(samples_data['Sample'])
    
    type_observed = tool
    
    data = samples_data.rename(columns={'frequency_TCR': 'frequency_truth'})
    data = data.rename(columns={'frequency_'+type_observed: 'frequency_observed'})
    data = data.rename(columns={'nReads_TCR': 'count_truth'})
    data = data.rename(columns={'nReads_'+type_observed: 'count_observed'})
    
    data = data.loc[data['frequency_truth'] > 0] #excluding clonotypes not present in ground truth

    file = open('../summary_data/original/capturing_per_sample/capturing_'+type_observed+'.csv',"w")
        
    file.write("th,observed_portion_frequency\n")
    
    data_per_sample = {}

    for s in samples:
        
        data_per_sample[s] = data.loc[data['Sample'] == s, ['frequency_truth','frequency_observed','count_truth','count_observed']]

    for th in np.linspace(0., 1.0, 100000):

        s_portions = []

        for s in samples: 

            k_observed=0
            k_true=0

            s_observed=0
            s_true=0

            sample_data = data_per_sample[s]
            sample_data = sample_data.loc[sample_data['frequency_truth'] >= th]
            
            for index, cdr in sample_data.iterrows():

                freq = cdr['frequency_truth']
                
                if cdr['frequency_observed'] > 0: #check if observed sample has the clonotype
                        
                    k_observed += 1.0    #number of clonotypes with frequency greater or equal than th
                    s_observed += freq   #sum of frequencies of observed greater or equal than th
                        
                k_true += 1.0 #number of clonotypes with frequency greater or equal than th
                s_true += freq #sum of frequencies of TCR greater or equal than th

            if s_true > 0:
                s_portions.append(s_observed/s_true)
            elif discard_zero_freq_samples is False:
                s_portions.append(0.)

        file.write(str(th)+","+str(np.mean(s_portions))+"\n")

    file.close()
    print('done '+tool)

Discarding samples with no clonotypes from truth to compute the average portion

In [4]:
# sample01

sample1 = data.loc[data['Sample'] == 'sample01']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample1, tool, True)

done MIXCR
done IMREP
done TRUST4


In [5]:
# sample02

sample2 = data.loc[data['Sample'] == 'sample02']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample2, tool, True)

done MIXCR
done IMREP
done TRUST4


In [6]:
# sample03

sample3 = data.loc[data['Sample'] == 'sample03']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample3, tool, True)

done MIXCR
done IMREP
done TRUST4


In [7]:
# sample04

sample4 = data.loc[data['Sample'] == 'sample04']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample4, tool, True)

done MIXCR
done IMREP
done TRUST4


In [8]:
# sample05

sample5 = data.loc[data['Sample'] == 'sample05']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample5, tool, True)

done MIXCR
done IMREP
done TRUST4


In [9]:
# sample06

sample6 = data.loc[data['Sample'] == 'sample06']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample6, tool, True)

done MIXCR
done IMREP
done TRUST4


In [10]:
# sample07

sample7 = data.loc[data['Sample'] == 'sample07']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample7, tool, True)

done MIXCR
done IMREP
done TRUST4


In [11]:
# sample08

sample8 = data.loc[data['Sample'] == 'sample08']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample8, tool, True)

done MIXCR
done IMREP
done TRUST4


In [12]:
# sample09

sample9 = data.loc[data['Sample'] == 'sample09']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample9, tool, True)

done MIXCR
done IMREP
done TRUST4


In [13]:
# sample10

sample10 = data.loc[data['Sample'] == 'sample10']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample10, tool, True)

done MIXCR
done IMREP
done TRUST4


In [14]:
# sample11

sample11 = data.loc[data['Sample'] == 'sample11']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample11, tool, True)

done MIXCR
done IMREP
done TRUST4


In [15]:
# sample12

sample12 = data.loc[data['Sample'] == 'sample12']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample12, tool, True)

done MIXCR
done IMREP
done TRUST4


In [16]:
# sample13

sample13 = data.loc[data['Sample'] == 'sample13']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample13, tool, True)

done MIXCR
done IMREP
done TRUST4


In [17]:
# sample14

sample14 = data.loc[data['Sample'] == 'sample14']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample14, tool, True)

done MIXCR
done IMREP
done TRUST4


In [18]:
# sample15

sample15 = data.loc[data['Sample'] == 'sample15']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample15, tool, True)

done MIXCR
done IMREP
done TRUST4


In [19]:
# SRR5233639

sample16 = data.loc[data['Sample'] == 'SRR5233639']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample16, tool, True)

done MIXCR
done IMREP
done TRUST4


In [20]:
# SRR5233637

sample17 = data.loc[data['Sample'] == 'SRR5233637']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample17, tool, True)

done MIXCR
done IMREP
done TRUST4


In [21]:
# TCGA-CZ-4862

sample18 = data.loc[data['Sample'] == 'TCGA-CZ-4862']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample18, tool, True)

done MIXCR
done IMREP
done TRUST4


In [22]:
# TCGA-CZ-5463

sample19 = data.loc[data['Sample'] == 'TCGA-CZ-5463']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample19, tool, True)

done MIXCR
done IMREP
done TRUST4


In [23]:
# TCGA-CZ-5985

sample20 = data.loc[data['Sample'] == 'TCGA-CZ-5985']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(sample20, tool, True)

done MIXCR
done IMREP
done TRUST4
